In [1]:
import os
import re
import html
import json
import argparse
import torch
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer

/home/s.zheng/sync/Gen_reco/.gen_venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
data = np.load('ID_generation/preprocessing/processed/Video_Games/Video_Games.blair-roberta-base.embeddings.npy')

In [12]:
data.shape

(115813, 768)

In [2]:
dataset = "Video_Games"
output_dir = f"./ID_generation/preprocessing/processed/{dataset}/"

train_file = f"{dataset}.train.inter"
valid_file = f"{dataset}.valid.inter"
test_file = f"{dataset}.test.inter"
datamap_files = f"{dataset}.data_maps"

In [3]:
# open the files and read the lines
train_lines = open(f"{output_dir}{train_file}", "r").readlines()
valid_lines = open(f"{output_dir}{valid_file}", "r").readlines()
test_lines = open(f"{output_dir}{test_file}", "r").readlines()

In [4]:
# Open datamap as a json file
with open(f"{output_dir}{datamap_files}", "r") as f:
    datamap = json.load(f)

# open All_Beauty.index.json

with open(f"./ID_generation/preprocessing/processed/{dataset}/{dataset}.index.json", "r") as f:
    id2code = json.load(f)

In [5]:
for key,value in id2code.items():
    new_value = ''.join(value)
    id2code[key] = new_value

In [6]:
def process_data(lines):
    lines_values = [line.strip().split("\t") for line in lines[1:]]

    train_interactions_user = [line[0] for line in lines_values]
    train_interactions_seq =  [line[1] for line in lines_values]
    train_interactions_target = [line[2] for line in lines_values]

    save_train_dict = []
    for user, seq, target in zip(train_interactions_user, train_interactions_seq, train_interactions_target):
        user_id = datamap['user2id'][user]
        seq = [datamap['item2id'][item] for item in seq.split(" ")]
        target = datamap['item2id'][target]
        seq.append(target)
        save_train_dict.append([user_id, seq])

    return save_train_dict

In [7]:
save_train_dict = process_data(train_lines)
save_valid_dict = process_data(valid_lines)
save_test_dict = process_data(test_lines)

In [8]:
len(save_train_dict), len(save_valid_dict), len(save_test_dict)

(1505033, 131348, 152463)

In [9]:
with open(f"./ID_generation/preprocessing/processed/{dataset}/{dataset}.train.inter.json", "w") as f:
    json.dump(save_train_dict, f)

with open(f"./ID_generation/preprocessing/processed/{dataset}/{dataset}.valid.inter.json", "w") as f:
    json.dump(save_valid_dict, f)

with open(f"./ID_generation/preprocessing/processed/{dataset}/{dataset}.test.inter.json", "w") as f:
    json.dump(save_test_dict, f)
